In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import sys
sys.path.insert(1, '../Config')
from lablib import *

## Funzioni di trasferimento

In [ ]:
def RC_R():
    def mod(W, R, C):
        return R/(np.sqrt(R**2 + 1/(W*C)**2))
    def fase(W, R, C):
        return np.arctan(1/(W*C*R))
    return mod, fase

def RC_C():

    def mod(W, R, C):
        return 1/(np.sqrt(1+(R*C*W)**2))
    def fase(W, R, C):
        return np.arctan(R*C*W)
    return mod, fase

def RL_R():
    def mod(W, R, L):
        return R/(R**2 + (W*L)**2)
    def fase(W, R, L):
        return np.arctan(W*L/R)
    return mod, fase

def RL_L():
    def mod(W, R, L):
        return W*L/(R**2 + (W*L)**2)
    def fase(W, R, L):
        return np.arctan(W*L/R)
    return mod, fase

def RLC_R():

    def mod(W, R, C, L):
        return R/(np.sqrt(R**2 + (W*L - 1/(W*C))**2))
    def fase(W, R, C, L):
        return np.arctan((W*L*C - 1)/(W*C*R))
    return mod, fase

def RLC_C():
    def mod(W, R, C, L):
        return 1/(np.sqrt((1 - (W**2)*L*C)**2 + (W*C*R)**2))
    def fase(W, R, C, L):
        return np.arctan(W*C*R/(1 - (W**2)*L*C))
    return mod, fase


def RLC_L():
    def mod(W, R, C, L):
        return W*L/(np.sqrt((1 - (W**2)*L*C)**2 + (W*C*R)**2))
    def fase(W, R, C, L):
        return np.arctan((1 - (W**2)*L*C)/(W*C*R))
    return mod, fase

# Prima parte RC RL 

# CIRCUITO RC
 Capacità utilizzata= $99 \pm 1$ nF \
 Resistenza= $100\pm 1 \Omega$ (check)
 
 Incertezza fase (CH2-CH1) = $\pm 1$ \
 Incertezza Ampiezza V_in(CH1) = $\pm 0.05$   Trasportarla su asse x e asse y \
 Incertezza Ampiezza V_out (CH2) = $\pm 0.05$ \
 Incertezza differenza (MATH)  = $\pm 0.01$ \

 le fasi nel foglio sono scritte in gradi \ 

 **R E C SONO SBAGLIATE**

In [ ]:
dati1 = pd.read_excel("dati_rc.xlsx")
RC_freq, RC_amp_A, RC_amp_B, RC_amp_ab, RC_r_fase_A, RC_r_fase_AB = dati1.iloc[:, 0:6].values.T
# .T serve a trasporre la matrice, ovvero a scambiare righe e colonne

### Resistenza

In [ ]:
# PLot dati RC resitenza

RC_H_R = [i/j for i,j in zip(RC_amp_B, RC_amp_A)]
RC_R_err = [0.05]*len(RC_H_R)

rc_r_mod, rc_r_fase = RC_R()
x = np.linspace(min(RC_freq), max(RC_freq), 1000)
Res = 100 #ohm
Cap = 100e-9 #farad

plt.errorbar(RC_freq, RC_H_R, RC_R_err, fmt = '.r', label = "Dati")
plt.plot(RC_freq, rc_r_mod(RC_freq, Res, Cap), label = "Modello")
plt.xscale('log')
plt.title("$H_R$ vs $\omega$")
plt.grid()
plt.legend()
plt.show()

In [ ]:
# fit dati resistenza

pars1, vals1, ers1, *other1 = LS_fit(RC_freq, RC_H_R, RC_R_err, rc_r_mod, R = Res, C = Cap)
# separare le funzioni sopra

In [ ]:
#plot resistenza fase

plt.errorbar(RC_freq, [np.deg2rad(i) for i in RC_r_fase_A], RC_R_err, fmt='.r', label='Dati')
plt.plot(x, rc_r_fase(x, Res, Cap), label='Modello')
plt.xscale('log')
plt.title("$\phi_R$ vs $\omega$")
plt.grid()
plt.legend()
plt.show()

In [ ]:
# fit resistenza fase

RC_F_err = [np.deg2rad(2)]*len(RC_r_fase_A)

pars2, vals2, ers2, *other2 = LS_fit(RC_freq, [np.deg2rad(i) for i in RC_r_fase_A], 
                                     RC_F_err, rc_r_fase, R = Res, C = Cap)


### Capacita

In [ ]:
#plot dati RC capacità

RC_H_C = [i/j for i,j in zip(RC_amp_ab, RC_amp_A)]
RC_C_err = [.05]*len(RC_H_C)
rc_c_mod, rc_c_fase = RC_C()

plt.errorbar(RC_freq, RC_H_C, RC_C_err, fmt='.r', label='Dati')
plt.plot(x, rc_c_mod(x, Res, Cap), label='Modello')
plt.xscale('log')
plt.legend()
plt.grid()
plt.show()

In [ ]:
# fit RC capacità

pars3, vals3, ers3, *other3 = LS_fit(RC_freq, RC_H_C, RC_C_err, rc_c_mod, R = Res, C = Cap)

In [ ]:
#plot RC capacità fase

plt.errorbar(RC_freq, [np.deg2rad(i) for i in RC_r_fase_AB], RC_F_err, fmt='.r', label='Dati')
plt.plot(x, rc_c_fase(x, Res, Cap), label='Modello')
plt.xscale('log')
plt.title("$\phi_C$ vs $\omega$")
plt.grid()
plt.legend()
plt.show()

# CIRCUITO RL


Induttanza utilizzata =   \
Resistenza utilizzata =   \
Incertezza fase =   \
Incertezza ampiezza V_in =  \ 
Incertezza ampiezza V_out =  \
Incertezza differenza (MATH) =   


# CIRCUITO RLC

Resistenza utilizzata = 
Induttanza utilizzata = 
Capacità utilizzata = 

